In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel':['linear','rbf','poly','sigmoid'],'C':[10,100,500,1000,2000,3000,5000,10000],'gamma':['auto','scale']}
grid = GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)     

Fitting 5 folds for each of 64 candidates, totalling 320 fits


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 500, 1000, 2000, 3000, 5000, 10000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [12]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)   
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [13]:
 from sklearn.metrics import f1_score
f1_macro = f1_score(y_test, grid_predictions, average = 'weighted')
print('The f1_macro value for best parameter {}'.format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'} 0.9100355779243318


In [14]:
print('The Confusion Matrix:\n',cm)

The Confusion Matrix:
 [[80  5]
 [ 7 42]]


In [15]:
print('The Report:\n',clf_report)

The Report:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93        85
           1       0.89      0.86      0.88        49

    accuracy                           0.91       134
   macro avg       0.91      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134



In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9539015606242497

In [17]:
table = pd.DataFrame.from_dict(re)

In [18]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.084382,0.065257,0.009376,0.012500,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,33
1,0.018751,0.006249,0.006251,0.007656,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,3
2,0.025001,0.007657,0.003125,0.006250,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,17
3,0.015624,0.000001,0.006251,0.007656,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.762677,0.738916,0.655795,0.796284,0.766556,0.744045,0.047743,63
4,0.031254,0.009883,0.003125,0.006251,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.015627,0.000002,0.003126,0.006252,10000,auto,sigmoid,"{'C': 10000, 'gamma': 'auto', 'kernel': 'sigmo...",0.791752,0.714931,0.698113,0.796284,0.766556,0.753527,0.040048,58
60,5.618207,2.103069,0.009377,0.007657,10000,scale,linear,"{'C': 10000, 'gamma': 'scale', 'kernel': 'line...",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,33
61,0.387589,0.073521,0.000000,0.000000,10000,scale,rbf,"{'C': 10000, 'gamma': 'scale', 'kernel': 'rbf'}",0.867478,0.886792,0.813179,0.847020,0.883278,0.859550,0.027112,11
62,12.876871,4.376582,0.006816,0.004811,10000,scale,poly,"{'C': 10000, 'gamma': 'scale', 'kernel': 'poly'}",0.822092,0.765553,0.766556,0.924528,0.901744,0.836095,0.066541,21


In [19]:
age_input=int(input("Age:"))
estimatedsalary_input=float(input("EstimatedSalary:"))
gender_male_input=int(input("Gender_Male 0 or 1:"))

Age:19
EstimatedSalary:19000
Gender_Male 0 or 1:1


In [20]:
Future_Prediction=grid.predict([[age_input,estimatedsalary_input,gender_male_input]])
print('Future_Prediction = {}'.format(Future_Prediction))

Future_Prediction = [1]
